# **Điểm chuẩn - Preprocessing**

Preprocessing data điểm chuẩn đã crawl về từ web [huongnghiep.hocmai.vn](https://huongnghiep.hocmai.vn/diem-chuan/)

## **Intro:**
- Bộ dữ liệu raw là các file csv như sau:  

    ```
    CRAWLER-HOCMAI  
    └── data  
        └── full  
            ├── diemchuan2018_demo.csv  
            ├── diemchuan2019_full.csv  
            ├── diemchuan2020_full.csv  
            ├── diemchuan2021_full.csv  
            ├── diemchuan2022_full.csv  
            ├── diemchuan2023_full.csv  
    ```
- Tương ứng mỗi file là dữ liệu điểm chuẩn của 1 năm.
    - 2 file `full` là dữ liệu đã được crawled đầy đủ.
    - 4 file `demo` là dữ liệu ***gần như*** đầy đủ, vì trong quá trình crawl có bị missing một vài trường. Sẽ cố gắng cập nhật đầy đủ sớm hơn.
- Cấu trúc dữ liệu mỗi file gồm 7 cột `university_code`, `university_name`, `major_code`, `major_name`, `subject_group`, `point`, `note`, `year`. 

    Một mẫu dữ liệu minh hoạ như sau: 
    
| university_code | university_name                 | major_code | major_name               | subject_group       | point | note             | year |
|-----------------|---------------------------------|------------|--------------------------|---------------------|-------|------------------|------|
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140234    | Sư phạm Tiếng Trung Quốc | D01, D15, D04, D45  | 24.85 | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140233    | Sư phạm Tiếng Pháp       | D01, D15, D03, D44  | 19    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220204    | Ngôn ngữ Trung Quốc      | D01, D15, D04, D45  | 23    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220210    | Ngôn ngữ Hàn Quốc        | D01, D14, D15       | 22.5  | Tốt nghiệp THPT  | 2023 |



## **Target**
- Kiểm tra tính đầy đủ của dữ liệu (về số trường đại học).

- Kiểm tra thiếu dữ liệu của từng dòng.

- Kiểm tra trùng lặp.

- **Phân tách dữ liệu cho từng loại điểm**


## **Start here**

### **import các thư viện**

In [75]:
import pandas as pd
import numpy as np
import os

### **Chỉ định đường dẫn thư mục**

In [76]:
# Directory containing the CSV files
directory = './data/full'


### **Đọc các file csv vào df**

In [77]:
# Dictionary to store DataFrames
dfs = {}
raw = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Store the DataFrame in the dictionary with the filename (without .csv) as the key
        dfs[filename.split('_')[0]] = df
        raw.append(df)

# Display the keys of the dictionary to confirm the DataFrames are loaded
dfs.keys()


dict_keys(['diemchuan2018', 'diemchuan2019', 'diemchuan2020', 'diemchuan2021', 'diemchuan2022', 'diemchuan2023'])

In [78]:
shapes = {name: df.shape for name, df in dfs.items()}
shapes

{'diemchuan2018': (4289, 8),
 'diemchuan2019': (5706, 8),
 'diemchuan2020': (4924, 8),
 'diemchuan2021': (6561, 8),
 'diemchuan2022': (7991, 8),
 'diemchuan2023': (7947, 8)}

In [79]:
full_df = pd.concat(raw, ignore_index=True)
full_df.head()

,university_code,university_name,major_code,major_name,subject_group,point,note,year
0,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018
1,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018
2,HUTECH,Đại học Công Nghệ TPHCM,7340120,Kinh doanh quốc tế,"A00, A01, D01, C00",19.0,NaN,2018
3,HUTECH,Đại học Công Nghệ TPHCM,7340201,Tài chính - Ngân hàng,"A00, A01, D01, C01",16.0,NaN,2018
4,HUTECH,Đại học Công Nghệ TPHCM,7340301,Kế toán,"A00, A01, D01, C01",16.0,NaN,2018


In [80]:
full_df.shape

(37418, 8)

In [81]:
uni_info = pd.read_csv("./data/university/university.csv")
uni_info.drop(columns='url',inplace=True)

In [82]:
full_df = pd.merge(full_df, uni_info, how="left", on=["university_code", "university_name"])
full_df

,university_code,university_name,major_code,major_name,subject_group,point,note,year,province,region,admission_code,institution_type
0,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
1,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
2,HUTECH,Đại học Công Nghệ TPHCM,7340120,Kinh doanh quốc tế,"A00, A01, D01, C00",19.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
3,HUTECH,Đại học Công Nghệ TPHCM,7340201,Tài chính - Ngân hàng,"A00, A01, D01, C01",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
4,HUTECH,Đại học Công Nghệ TPHCM,7340301,Kế toán,"A00, A01, D01, C01",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37413,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37414,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37415,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37416,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập


#### Đổi tên các cột lại thành tiếng Việt

In [83]:
# Dictionary to map old column names to new column names
column_mapping = {
    'province': 'Tỉnh/ Thành phố',
    'region': "Khu vực",
    'admission_code': 'Mã xét tuyển',
    'university_code': 'Mã Trường',
    'university_name': 'Tên Trường',
    'major_code': 'Mã Ngành',
    'major_name': 'Tên Ngành',
    'subject_group': 'Tổ hợp',
    'point': 'Điểm chuẩn',
    'note': 'Ghi chú',
    'year': 'Năm',
    'institution_type': 'Loại đơn vị'
}

# Rename columns for each DataFrame in the dictionary
for name, df in dfs.items():
    df.rename(columns=column_mapping, inplace=True)

full_df.rename(columns=column_mapping, inplace=True)
full_df.head()

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Loại đơn vị
0,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
1,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
2,HUTECH,Đại học Công Nghệ TPHCM,7340120,Kinh doanh quốc tế,"A00, A01, D01, C00",19.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
3,HUTECH,Đại học Công Nghệ TPHCM,7340201,Tài chính - Ngân hàng,"A00, A01, D01, C01",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập
4,HUTECH,Đại học Công Nghệ TPHCM,7340301,Kế toán,"A00, A01, D01, C01",16.0,NaN,2018,Hồ Chí Minh,Miền Nam,DKC,Dân lập


In [84]:
full_df = full_df[[ 'Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển','Mã Trường', 'Tên Trường','Mã Ngành', 'Tên Ngành', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm', 'Loại đơn vị']]

In [85]:
full_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
0,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập
1,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Dân lập
2,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340120,Kinh doanh quốc tế,"A00, A01, D01, C00",19.0,NaN,2018,Dân lập
3,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340201,Tài chính - Ngân hàng,"A00, A01, D01, C01",16.0,NaN,2018,Dân lập
4,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340301,Kế toán,"A00, A01, D01, C01",16.0,NaN,2018,Dân lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37413,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37414,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37415,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37416,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập


### **Khám phá dữ liệu**

#### 1. Mỗi dòng có ý nghĩa gì?

In [86]:
full_df.sample(5)

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
26772,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7580102,Kiến trúc,"A00, D01, V00, V01, XDHB",0.00,NaN,2022,Công lập
7772,Hà Nội,Miền Bắc,BPH,HVBP,Học Viện Biên Phòng,7380101,Luật,A01,23.65,Luật Thí sinh Nam Quân khu 4 (Quảng Trị và TT-...,2019,Công lập
8971,Hồ Chí Minh,Miền Nam,SPS,HCMUE,Đại Học Sư Phạm TPHCM,7140219,Sư phạm Địa lý,"C00, C04, D78",28.00,NaN,2019,Công lập
12969,Cần Thơ,Miền Nam,TCT,CTU,Đại Học Cần Thơ,7850102,Kinh tế,"A00, A01, D01, C02",21.50,Kinh tế tài nguyên thiên nhiên,2020,Công lập
22372,Cần Thơ,Miền Nam,TCT,CTU,Đại Học Cần Thơ,7140217,Sư phạm Ngữ văn,"C00, D14, D15",26.50,Điểm thi TN THPT,2022,Công lập


Mỗi dòng là thông tin về điểm chuẩn của một ngành, tại một trường, vào một năm cụ thể.

##### Kiểm tra trùng lặp

In [87]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Tồn tại 14 dòng bị trùng lặp trong dữ liệu.


Xuất các dòng trùng lặp ra file csv (Để coi chơi thôi chứ cũng drop à :v)

In [88]:
full_df[full_df.duplicated()].to_csv('./data/dup.csv',  encoding='utf-8-sig')

Loại bỏ các dòng trùng

In [89]:
full_df.drop_duplicates(inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_107468\2742331558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop_duplicates(inplace=True)


Kiểm tra lại

In [90]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Không có dòng nào trùng lặp trong dữ liệu.


#### 2. Mỗi cột có ý nghĩa gì?

Các cột và thông tin tương ứng:
- `Mã Trường`: Tên viết tắt của trường (Không phải mã trên hệ thống của bộ).

- `Tên Trường`: Tên của Trường đại học.

- `	Mã Ngành`: Mã ngành tuyển sinh.

- `	Tên Ngành`: Tên ngành tuyển sinh.

- `	Tổ hợp`: Các tổ hợp môn xét tuyển.

- `	Điểm chuẩn`: Điểm chuẩn tương ứng với tổ hợp.

- `	Ghi chú`: Ghi chú thêm (thường là thông tin về điều kiện bổ sung, hoặc phương thức xét tuyển).

- `	Năm`: Năm tuyển sinh.

#### 3. Mỗi cột hiện có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần xử lý tiếp hay không?

In [91]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37404 entries, 0 to 37417
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tỉnh/ Thành phố  37404 non-null  object 
 1   Khu vực          37404 non-null  object 
 2   Mã xét tuyển     37404 non-null  object 
 3   Mã Trường        37404 non-null  object 
 4   Tên Trường       37404 non-null  object 
 5   Mã Ngành         37404 non-null  object 
 6   Tên Ngành        37404 non-null  object 
 7   Tổ hợp           37381 non-null  object 
 8   Điểm chuẩn       37404 non-null  float64
 9   Ghi chú          25072 non-null  object 
 10  Năm              37404 non-null  int64  
 11  Loại đơn vị      37404 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 3.7+ MB


Các cột có kiểu dữ liệu phù hợp.

#### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

##### 4.1. Các cột dạng số

In [92]:
numerical_columns = full_df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical columns:", numerical_columns)

def calculate_missing_ratio(df):
    # Tính tỷ lệ dữ liệu thiếu cho từng cột
    missing_ratios = df.isna().mean() * 100

    # Trả về Series chứa tỷ lệ dữ liệu thiếu cho từng cột
    return missing_ratios

# Gọi hàm tính missing ratio và in kết quả
missing_ratios = calculate_missing_ratio(full_df[numerical_columns])
print('Missing ratio:')
print(missing_ratios)
full_df.describe()

Numerical columns: Index(['Điểm chuẩn', 'Năm'], dtype='object')
Missing ratio:
Điểm chuẩn    0.0
Năm           0.0
dtype: float64


,Điểm chuẩn,Năm
count,37404.000000,37404.000000
mean,74.306515,2020.857341
std,183.169403,1.678110
min,0.000000,2018.000000
25%,16.000000,2019.000000
50%,20.000000,2021.000000
75%,25.100000,2022.000000
max,1035.000000,2023.000000


In [93]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


In [94]:
full_df.drop(missing_rows.index, inplace=True)
full_df.reset_index(drop=True, inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_107468\3106419298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(missing_rows.index, inplace=True)


In [95]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


Thấy min của điểm chuẩn là 0 (kỳ lạ) nên ta sẽ xử lý các dòng có điểm chuẩn = 0 một chút.


In [96]:
full_df[full_df['Điểm chuẩn']==0]

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
322,Hồ Chí Minh,Miền Nam,DTT,TDTU,Đại Học Tôn Đức Thắng,K7340120,Quản trị kinh doanh,"A00, A01, D01",0.0,Quản trị kinh doanh quốc tế (đơn bằng 3+1) - C...,2018,Công lập
398,Hà Nội,Miền Bắc,XDA,NUCE,Đại Học Xây Dựng Hà Nội,7580201_04,Kỹ thuật xây dựng,"A00, A01, D07",0.0,Kỹ thuật xây dựng Công trình thủy (chuyên ngàn...,2018,Công lập
408,Hồ Chí Minh,Miền Nam,DTT,TDTU,Đại Học Tôn Đức Thắng,K7520201,"Kỹ thuật điện, điện tử","A00, A01, D01",0.0,"Kỹ thuật điện – điện tử (song bằng 2,5+1,5) - ...",2018,Công lập
590,Đồng Nai,Miền Nam,DNU,DNU,Đại Học Đồng Nai,51140206,Giáo dục Thể chất,"T00, T04",0.0,hệ cao đẳng,2018,Công lập
592,Đồng Nai,Miền Nam,DNU,DNU,Đại Học Đồng Nai,51140221,Sư phạm Âm nhạc,"N00, N01",0.0,Hệ cao đẳng,2018,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37041,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37043,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37045,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37047,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


Ta thấy có rất nhiều ngành có điểm chuẩn =0, tuy nhiên, một số sẽ có chú thích riêng. Vậy, đối với các ngành có điểm chuẩn =0 và không có chú thích thì mình xem như năm đó không có tuyển sinh. 
> => Mình loại bỏ các dòng này.

In [97]:
diemchuan_0 = full_df[(full_df['Điểm chuẩn']==0) & (full_df['Ghi chú'].isna())]
diemchuan_0

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
1303,Hà Nội,Miền Bắc,XDA,NUCE,Đại Học Xây Dựng Hà Nội,7480101,Kỹ thuật xây dựng,"A00, A01, D07",0.0,NaN,2018,Công lập
1307,Hà Nội,Miền Bắc,XDA,NUCE,Đại Học Xây Dựng Hà Nội,7520103,Kỹ thuật cơ khí,"A00, A01, D07",0.0,NaN,2018,Công lập
1308,Hà Nội,Miền Bắc,XDA,NUCE,Đại Học Xây Dựng Hà Nội,7520320,Kỹ thuật môi trường,"A00, B00, D07",0.0,NaN,2018,Công lập
1341,Hồ Chí Minh,Miền Nam,DTT,TDTU,Đại Học Tôn Đức Thắng,7340115,Marketing,"A00, A01, D01",0.0,NaN,2018,Công lập
1347,Hồ Chí Minh,Miền Nam,DTT,TDTU,Đại Học Tôn Đức Thắng,7810302,Golf,"A01, D01, T00, T01",0.0,NaN,2018,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37041,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37043,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37045,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37047,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


In [98]:
shape = full_df.shape

# Drop các dòng có trong diemchuan_0
full_df.drop(diemchuan_0.index, inplace=True)

# Reset lại chỉ số sau khi drop
full_df.reset_index(drop=True, inplace=True)

print(f"Dữ liệu trước xử lý có {shape[0]} dòng và {shape[1]} cột.")
print(f"Dữ liệu   sau xử lý có {full_df.shape[0]} dòng và {full_df.shape[1]} cột.")

Dữ liệu trước xử lý có 37404 dòng và 12 cột.
Dữ liệu   sau xử lý có 36737 dòng và 12 cột.


C:\Users\letra\AppData\Local\Temp\ipykernel_107468\1664910539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(diemchuan_0.index, inplace=True)


##### 4.2. Các cột dạng phân loại

In [99]:
categorical_columns = full_df.select_dtypes(include=['object']).columns
categorical_columns

Index(['Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển', 'Mã Trường', 'Tên Trường',
       'Mã Ngành', 'Tên Ngành', 'Tổ hợp', 'Ghi chú', 'Loại đơn vị'],
      dtype='object')

In [100]:
def calculate_column_profile(column):
    missing_ratio = len(column[column.isna()]) * 100 / len(column)
    num_diff_vals = len(column.dropna().unique())
    diff_vals = column.dropna().unique()
    return pd.Series([missing_ratio, num_diff_vals, diff_vals], index=['missing_ratio', 'num_diff_vals', 'diff_vals'])

cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01361,31.752729,0.0
num_diff_vals,52,3,277,256,289,2598,677,2494,5265,2
diff_vals,"[Hồ Chí Minh, Quảng Bình, Bạc Liêu, Tiền Giang...","[Miền Nam, Miền Trung - Tây Nguyên, Miền Bắc]","[DKC, DQB, DBL, TTG, DTT, GHA, GTA, DDF, DPQ, ...","[HUTECH, QBU, BLU, TGU, TDTU, UTC, UTT, UFLS, ...","[Đại học Công Nghệ TPHCM, Đại Học Quảng Bình, ...","[7310608, 7340115, 7340120, 7340201, 7340301, ...","[Đông phương học, Marketing, Kinh doanh quốc t...","[A01, D01, C00, D15, A00, A01, D01, C00, A00, ...","[Xét tuyển học bạ lấy 18.0 điểm, Xét tuyển học...","[Dân lập, Công lập]"


(Lại) Thấy `Tổ hợp` có missing, trong khi dữ liệu này đáng lẽ không nên miss.  Mình in ra xem thử

In [101]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Mã Ngành' hoặc cột 'Tổ hợp' (nếu có)
missing_rows = full_df[full_df['Tổ hợp'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
2611,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6810201,Quản trị khách sạn,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
2612,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6810103,Hướng dẫn du lịch,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
2613,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6220213,Tiếng Nhật,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
2614,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6220206,Tiếng Anh,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
4865,Hà Nội,Miền Bắc,HVN,VNUA,Học Viện Nông Nghiệp Việt Nam,7140215,Sư phạm Kỹ thuật nông nghiệp,NaN,0.0,Tuyển thẳng đối với thí sinh đạt học lực giỏi ...,2019,Công lập


Thấy toàn bộ missing là tuyển thẳng, nên Replace "Tuyển thẳng"

In [102]:
full_df.loc[full_df['Tổ hợp'].isna(), 'Tổ hợp'] = 'Tuyển thẳng'

In [103]:
cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.752729,0.0
num_diff_vals,52,3,277,256,289,2598,677,2495,5265,2
diff_vals,"[Hồ Chí Minh, Quảng Bình, Bạc Liêu, Tiền Giang...","[Miền Nam, Miền Trung - Tây Nguyên, Miền Bắc]","[DKC, DQB, DBL, TTG, DTT, GHA, GTA, DDF, DPQ, ...","[HUTECH, QBU, BLU, TGU, TDTU, UTC, UTT, UFLS, ...","[Đại học Công Nghệ TPHCM, Đại Học Quảng Bình, ...","[7310608, 7340115, 7340120, 7340201, 7340301, ...","[Đông phương học, Marketing, Kinh doanh quốc t...","[A01, D01, C00, D15, A00, A01, D01, C00, A00, ...","[Xét tuyển học bạ lấy 18.0 điểm, Xét tuyển học...","[Dân lập, Công lập]"


In [104]:
full_df[full_df['Tỉnh/ Thành phố'].isna()].to_csv('test.csv',  encoding='utf-8-sig', index=False)

### **Tách cột Tổ hợp**

In [105]:
# Tách cột "Tổ hợp" theo dấu phẩy và tạo các dòng mới
#df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('Tổ hợp')
df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('subject_group')

df_expanded.reset_index(drop=True, inplace=True)
df_expanded

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,subject_group
0,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,A01
1,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D01
2,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,C00
3,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D15
4,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Dân lập,A00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114331,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
114332,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
114333,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
114334,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM


In [106]:
condition2 = (df_expanded['Ghi chú'].str.contains('ao đẳng', na=False))
df_expanded.drop(df_expanded[condition2].index, inplace=True)

# Lọc và xóa các dòng có subject_group là 'XDHB' và 'ọc bạ' tồn tại trong Ghi chú
condition = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('HỌC BẠ', na=False))
df_expanded.drop(df_expanded[condition].index, inplace=True)


condition3 = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Tổ hợp'].str.contains(', XDHB', na=False))
df_expanded.drop(df_expanded[condition3].index, inplace=True)

# Đặt lại chỉ mục của full_df và lưu thay đổi này vào DataFrame gốc
df_expanded.reset_index(drop=True, inplace=True)
df_expanded



,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,subject_group
0,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,A01
1,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D01
2,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,C00
3,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D15
4,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Dân lập,A00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107450,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
107451,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
107452,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM
107453,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM


In [107]:
categorical_columns = df_expanded.select_dtypes(include=['object']).columns
cate_col_profiles_df = df_expanded[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị,subject_group
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.307059,0.0,0.0
num_diff_vals,52,3,277,256,289,2582,676,2488,5169,2,249
diff_vals,"[Hồ Chí Minh, Quảng Bình, Bạc Liêu, Tiền Giang...","[Miền Nam, Miền Trung - Tây Nguyên, Miền Bắc]","[DKC, DQB, DBL, TTG, DTT, GHA, GTA, DDF, DPQ, ...","[HUTECH, QBU, BLU, TGU, TDTU, UTC, UTT, UFLS, ...","[Đại học Công Nghệ TPHCM, Đại Học Quảng Bình, ...","[7310608, 7340115, 7340120, 7340201, 7340301, ...","[Đông phương học, Marketing, Kinh doanh quốc t...","[A01, D01, C00, D15, A00, A01, D01, C00, A00, ...","[Xét tuyển học bạ lấy 18.0 điểm, Xét tuyển học...","[Dân lập, Công lập]","[A01, D01, C00, D15, A00, C01, B00, D07, C08, ..."


In [108]:
# df_expanded.to_csv("./data/preprocessing/full.csv",  encoding='utf-8-sig', index=False)

In [109]:
# Tạo cột 'Loại điểm' dựa trên 'Ghi chú' và 'Tổ hợp'
df_expanded['Loại điểm'] = ''
df_expanded.loc[(df_expanded['Ghi chú']=='Xét học bạ')|(df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['Tổ hợp'].str.contains('XDHB', na=False)), 'Loại điểm'] = 'Xét học bạ'

# Cập nhật 'Loại điểm' dựa trên 'Tổ hợp'
df_expanded.loc[((df_expanded['Tổ hợp'].str.contains('DGNL', na=False)) | (df_expanded['Tổ hợp'].str.contains('DGTD', na=False))), 'Loại điểm'] = 'Kỳ thi riêng'
df_expanded.loc[df_expanded['Loại điểm'] == '', 'Loại điểm'] = 'THPTQG'

df_expanded

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,subject_group,Loại điểm
0,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,A01,THPTQG
1,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D01,THPTQG
2,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,C00,THPTQG
3,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,"A01, D01, C00, D15",16.0,NaN,2018,Dân lập,D15,THPTQG
4,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,"A00, A01, D01, C00",20.0,NaN,2018,Dân lập,A00,THPTQG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107450,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM,Kỳ thi riêng
107451,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM,Kỳ thi riêng
107452,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM,Kỳ thi riêng
107453,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,DGNLHCM,Kỳ thi riêng


In [110]:
df_expanded.drop(columns = ['Tổ hợp'], inplace=True)
df_expanded.rename(columns={'subject_group': 'Tổ hợp'}, inplace=True)
df_expanded = df_expanded[['Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển','Mã Trường', 'Tên Trường', 'Mã Ngành', 'Tên Ngành', 'Loại điểm', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm']]
df_expanded 

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Loại điểm,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
0,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,THPTQG,A01,16.0,NaN,2018
1,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,THPTQG,D01,16.0,NaN,2018
2,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,THPTQG,C00,16.0,NaN,2018
3,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7310608,Đông phương học,THPTQG,D15,16.0,NaN,2018
4,Hồ Chí Minh,Miền Nam,DKC,HUTECH,Đại học Công Nghệ TPHCM,7340115,Marketing,THPTQG,A00,20.0,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
107450,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,Kỳ thi riêng,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107451,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,Kỳ thi riêng,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107452,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,Kỳ thi riêng,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107453,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,Kỳ thi riêng,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023


In [111]:
df_expanded.to_csv("./data/preprocessing/diemchuan_full.csv",  encoding='utf-8-sig', index=False)